# 探索 OpenAI V1 功能

在 2023 年 11 月 6 日，OpenAI 发布了许多新功能，并将其 Python SDK 版本升级到 1.0.0。本笔记本展示了这些新功能以及如何在 LangChain 中使用它们。

In [ ]:
# 需要 openai>=1.1.0, langchain>=0.0.335, langchain-experimental>=0.0.39
!pip install -U openai langchain langchain-experimental

In [1]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

## [视觉功能](https://platform.openai.com/docs/guides/vision)

OpenAI 发布了多模态模型，可以接收文本和图像序列作为输入。

In [ ]:
chat = ChatOpenAI(model="gpt-4-vision-preview", max_tokens=256)
chat.invoke(
    [
        HumanMessage(
            content=[
                {"type": "text", "text": "这张图片显示了什么"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/static/img/langchain_stack.png",
                        "detail": "auto",
                    },
                },
            ]
        )
    ]
)

AIMessage(content='The image appears to be a diagram representing the architecture or components of a software system or framework related to language processing, possibly named LangChain or associated with a project or product called LangChain, based on the prominent appearance of that term. The diagram is organized into several layers or aspects, each containing various elements or modules:\n\n1. **Protocol**: This may be the foundational layer, which includes "LCEL" and terms like parallelization, fallbacks, tracing, batching, streaming, async, and composition. These seem related to communication and execution protocols for the system.\n\n2. **Integrations Components**: This layer includes "Model I/O" with elements such as the model, output parser, prompt, and example selector. It also has a "Retrieval" section with a document loader, retriever, embedding model, vector store, and text splitter. Lastly, there\'s an "Agent Tooling" section. These components likely deal with the intera

## [OpenAI 助手](https://platform.openai.com/docs/assistants/overview)

> Assistants API 允许您在自己的应用程序中构建 AI 助手。助手具有指令，可以利用模型、工具和知识来响应用户查询。Assistants API 目前支持三种类型的工具：代码解释器、检索和函数调用。

您可以使用 OpenAI 工具或自定义工具与 OpenAI 助手进行交互。当仅使用 OpenAI 工具时，您可以直接调用助手并获取最终答案。当使用自定义工具时，您可以使用内置的 AgentExecutor 运行助手和工具执行循环，或者轻松编写自己的执行器。

下面我们展示与助手交互的不同方式。作为一个简单的示例，让我们构建一个可以编写和运行代码的数学导师。

### 仅使用 OpenAI 工具

In [1]:
from langchain.agents.openai_assistant import OpenAIAssistantRunnable

In [ ]:
interpreter_assistant = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant",
    instructions="你是一个私人数学导师。编写并运行代码以回答数学问题。",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview",
)
output = interpreter_assistant.invoke({"content": "10 - 4 的 2.7 次方是多少"})
output

[ThreadMessage(id='msg_g9OJv0rpPgnc3mHmocFv7OVd', assistant_id='asst_hTwZeNMMphxzSOqJ01uBMsJI', content=[MessageContentText(text=Text(annotations=[], value='The result of \\(10 - 4^{2.7}\\) is approximately \\(-32.224\\).'), type='text')], created_at=1699460600, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_nBIT7SiAwtUfSCTrQNSPLOfe', thread_id='thread_14n4GgXwxgNL0s30WJW5F6p0')]

### 作为具有任意工具的 LangChain 代理

现在让我们使用我们自己的工具重新创建这个功能。对于这个示例，我们将使用 [E2B 沙箱运行时工具](https://e2b.dev/docs?ref=landing-page-get-started)。

In [ ]:
!pip install e2b duckduckgo-search

In [3]:
from langchain.tools import DuckDuckGoSearchRun, E2BDataAnalysisTool

tools = [E2BDataAnalysisTool(api_key="..."), DuckDuckGoSearchRun()]

In [ ]:
agent = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant e2b tool",
    instructions="你是一个私人数学导师。编写并运行代码以回答数学问题。你还可以搜索互联网。",
    tools=tools,
    model="gpt-4-1106-preview",
    as_agent=True,
)

#### 使用 AgentExecutor

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)
agent_executor.invoke({"content": "今天旧金山的天气除以 2.7 是多少"})

{'content': "What's the weather in SF today divided by 2.7",
 'output': "The weather in San Francisco today is reported to have temperatures as high as 66 °F. To get the temperature divided by 2.7, we will calculate that:\n\n66 °F / 2.7 = 24.44 °F\n\nSo, when the high temperature of 66 °F is divided by 2.7, the result is approximately 24.44 °F. Please note that this doesn't have a meteorological meaning; it's purely a mathematical operation based on the given temperature."}

#### 自定义执行

In [ ]:
agent = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant e2b tool",
    instructions="你是一个私人数学导师。编写并运行代码以回答数学问题。",
    tools=tools,
    model="gpt-4-1106-preview",
    as_agent=True,
)

In [7]:
from langchain_core.agents import AgentFinish


def execute_agent(agent, tools, input):
    tool_map = {tool.name: tool for tool in tools}
    response = agent.invoke(input)
    while not isinstance(response, AgentFinish):
        tool_outputs = []
        for action in response:
            tool_output = tool_map[action.tool].invoke(action.tool_input)
            print(action.tool, action.tool_input, tool_output, end="\n\n")
            tool_outputs.append(
                {"output": tool_output, "tool_call_id": action.tool_call_id}
            )
        response = agent.invoke(
            {
                "tool_outputs": tool_outputs,
                "run_id": action.run_id,
                "thread_id": action.thread_id,
            }
        )

    return response

In [ ]:
response = execute_agent(agent, tools, {"content": "10 - 4 的 2.7 次方是多少"})
print(response.return_values["output"])

e2b_data_analysis {'python_code': 'print(10 - 4 ** 2.7)'} {"stdout": "-32.22425314473263", "stderr": "", "artifacts": []}

\( 10 - 4^{2.7} \) is approximately \(-32.22425314473263\).


In [ ]:
next_response = execute_agent(
    agent, tools, {"content": "现在加上 17.241", "thread_id": response.thread_id}
)
print(next_response.return_values["output"])

e2b_data_analysis {'python_code': 'result = 10 - 4 ** 2.7\nprint(result + 17.241)'} {"stdout": "-14.983253144732629", "stderr": "", "artifacts": []}

When you add \( 17.241 \) to \( 10 - 4^{2.7} \), the result is approximately \( -14.98325314473263 \).


## [JSON 模式](https://platform.openai.com/docs/guides/text-generation/json-mode)

限制模型只生成有效的 JSON。注意，您必须包含一个带有使用 JSON 指令的系统消息，才能使此模式工作。

仅适用于某些模型。

In [ ]:
chat = ChatOpenAI(model="gpt-3.5-turbo-1106").bind(
    response_format={"type": "json_object"}
)

output = chat.invoke(
    [
        SystemMessage(
            content="提取以下陈述中提到的任何公司的 'name' 和 'origin'。返回一个 JSON 列表。"
        ),
        HumanMessage(
            content="Google 成立于美国，而 Deepmind 成立于英国"
        ),
    ]
)
print(output.content)

In [ ]:
import json

json.loads(output.content)

## [系统指纹](https://platform.openai.com/docs/guides/text-generation/reproducible-outputs)

OpenAI 有时会以影响输出的方式更改模型配置。每当这种情况发生时，与生成相关的 system_fingerprint 将会改变。

In [ ]:
chat = ChatOpenAI(model="gpt-3.5-turbo-1106")
output = chat.generate(
    [
        [
            SystemMessage(
                content="提取以下陈述中提到的任何公司的 'name' 和 'origin'。返回一个 JSON 列表。"
            ),
            HumanMessage(
                content="Google 成立于美国，而 Deepmind 成立于英国"
            ),
        ]
    ]
)
print(output.llm_output)

## Azure 类的重大变更

OpenAI V1 重写了他们的客户端，并分离了 Azure 和 OpenAI 客户端。这导致在使用 OpenAI V1 时 LangChain 接口发生了一些变化。

重大变更：
- 要使用带有 OpenAI V1 的 Azure 嵌入，您需要使用新的 `AzureOpenAIEmbeddings` 而不是现有的 `OpenAIEmbeddings`。当使用 `openai<1` 的 Azure 时，`OpenAIEmbeddings` 继续工作。
```python
from langchain_openai import AzureOpenAIEmbeddings
```

建议的变更：
- 当使用 `AzureChatOpenAI` 或 `AzureOpenAI` 时，如果传入 Azure 端点（例如 https://example-resource.azure.openai.com/），应通过 `azure_endpoint` 参数或 `AZURE_OPENAI_ENDPOINT` 指定。我们暂时保持通过 `openai_api_base`/`base_url` 或环境变量 `OPENAI_API_BASE` 指定的向后兼容性，但不应依赖这种方式。
- 使用 Azure 聊天或嵌入模型时，通过 `openai_api_key` 参数或 `AZURE_OPENAI_API_KEY` 参数传入 API 密钥。我们暂时保持通过 `OPENAI_API_KEY` 指定的向后兼容性，但不应依赖这种方式。

## 工具

使用工具进行并行函数调用。

In [ ]:
from typing import Literal

from langchain.output_parsers.openai_tools import PydanticToolsParser
from langchain.utils.openai_functions import convert_pydantic_to_openai_tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field


class GetCurrentWeather(BaseModel):
    """获取某地的当前天气。"""

    location: str = Field(description="城市和州，例如 San Francisco, CA")
    unit: Literal["celsius", "fahrenheit"] = Field(
        default="fahrenheit", description="温度单位，默认为华氏度"
    )


prompt = ChatPromptTemplate.from_messages(
    [("system", "你是一个乐于助人的助手"), ("user", "{input}")]
)
model = ChatOpenAI(model="gpt-3.5-turbo-1106").bind(
    tools=[convert_pydantic_to_openai_tool(GetCurrentWeather)]
)
chain = prompt | model | PydanticToolsParser(tools=[GetCurrentWeather])

chain.invoke({"input": "纽约、洛杉矶和旧金山的天气怎么样"})

[GetCurrentWeather(location='New York, NY', unit='fahrenheit'),
 GetCurrentWeather(location='Los Angeles, CA', unit='fahrenheit'),
 GetCurrentWeather(location='San Francisco, CA', unit='fahrenheit')]